<a href="https://colab.research.google.com/github/jeffreyestayobasanes/Mia/blob/main/Mia_(Virtual_Assistant).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instruction: Always click the 'Play/Run' button in every part of the source code.

In [12]:
# Machines don't have these and should be installed first
# These are all packages necessary to have access to microphone and audio hardware
# Run these first so the functions to play audio and listen to microphones can be accessible and used

!pip install ffmpeg-python # Allows Google Colab to have access to microphone
!pip install SpeechRecognition
!pip install datetime
!pip install playsound
!pip install gTTs
!apt-get install portaudio19-dev
!pip3 install pyaudio

!pip install numpy
!pip install matplotlib
!pip install comtypes
!pip install opencv-python

!pip install PyDictionary

!pip install beautifulsoup
!pip install requests

Reading package lists... Done
Building dependency tree       
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.
  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Using cached BeautifulSoup-3.2.1.tar.gz (31 kB)
  Using cached BeautifulSoup-3.2.0.tar.gz (31 kB)
ERROR: Could not find a version that satisfies the requirement beautifulsoup (from versions: 3.2.0, 3.2.1, 3.2.2)
ERROR: No matching distribution found for beautifulsoup


In [13]:

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var mylabel = document.createElement("LABEL");
var mybutton = document.createElement("BUTTON");
var my_btn = document.createElement("BUTTON");
mybutton.style.width = "300px"; 
mybutton.style.height = "300px"; 
mybutton.style.backgroundImage = "url(https://raw.githubusercontent.com/jeffreyestayobasanes/Mia/main/sleeping.png)"; 
mybutton.style.backgroundSize = "cover";

my_btn.style.width = "300px"; 
my_btn.style.height = "300px"; 
my_btn.style.backgroundImage = "url(https://raw.githubusercontent.com/jeffreyestayobasanes/Mia/main/listening.png)"; 
my_btn.style.backgroundSize = "cover";

my_div.appendChild(mybutton);
my_div.appendChild(my_btn);
my_div.appendChild(mylabel);
document.body.appendChild(my_div);
var recordButton = my_btn;
var base64data = 0;
var reader;
var recorder, gumStream;
mybutton.style.display = "block";
my_btn.style.display = "none";


mybutton.onclick = ()=>{   

    
    var handleSuccess = function(stream) {
    gumStream = stream;
    var options = {
      //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
      mimeType : 'audio/webm;codecs=opus'
      //mimeType : 'audio/webm;codecs=pcm'
    };   

    //recorder = new MediaRecorder(stream, options);
    recorder = new MediaRecorder(stream);
    recorder.ondataavailable = function(e) {            
      var url = URL.createObjectURL(e.data);
      var preview = document.createElement('audio');
      preview.controls = true;
      preview.src = url;
      //document.body.appendChild(preview);

      reader = new FileReader();
      reader.readAsDataURL(e.data); 
      reader.onloadend = function() {
        base64data = reader.result;
        //console.log("Inside FileReader:" + base64data);
      }
    };
    recorder.start();
  };
  mybutton.style.display = "none";
  my_btn.style.display = "block";


  navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

}

  var data = new Promise(resolve=>{
  //recordButton.addEventListener("click", toggleRecording);
  recordButton.onclick = ()=>{
  toggleRecording()

  sleep(100).then(() => {
    // wait 2000ms for the data to be available...
    // ideally this should use something like await...
    //console.log("Inside data:" + base64data)
    resolve(base64data.toString())

  });

  }
  }); //var data

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();      
      sleep(500).then(() => {
            recordButton.style.display = "none";
            recordButton.innerText = "Input Accepted"
      });
      
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

</script>
"""


MANUAL_HTML = """
<script>
  
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var mylabel = document.createElement("LABEL");
var mybutton = document.createElement("A");
var my_btn = document.createElement("A");
mybutton.style.width = "300px"; 
mybutton.style.height = "300px"; 
mybutton.setAttribute("target", "_blank");
mybutton.href="https://raw.githubusercontent.com/jeffreyestayobasanes/Mia/main/Mia_Manual.png";
mybutton.style.backgroundImage = "url(https://raw.githubusercontent.com/jeffreyestayobasanes/Mia/main/Mia_Manual_Cover.png)"; 
mybutton.style.backgroundSize = "cover";

mybutton.onclick = function() {
    mybutton.style.display="none";
}

my_btn.style.width = "300px"; 
my_btn.style.height = "300px"; 
my_btn.style.backgroundImage = "url(https://raw.githubusercontent.com/jeffreyestayobasanes/Mia/main/listening.png)"; 
mybutton.style.backgroundSize = "cover";

my_div.appendChild(mybutton);
my_div.appendChild(my_btn);
my_div.appendChild(mylabel);
document.body.appendChild(my_div);
var recordButton = my_btn;
mybutton.style.display = "block";
my_btn.style.display = "none";
sleep(1200).then(() => {
  mybutton.style.display = "block";
});
</script>
"""

In [95]:
import speech_recognition as sr
import random
from gtts import gTTS
from IPython.display import Audio, display
def get_audio_wav():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
    
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
    
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  audio2 = io.BytesIO(riff)
  return audio2

def manualLink():
  display(HTML(MANUAL_HTML))

def get_audio():    
    r = sr.Recognizer()
    audio2 = sr.WavFile(get_audio_wav())
    with audio2 as source:              
        audio = r.record(source)        # extract audio data from the file
        said = ""                        
        try:
            said = r.recognize_google(audio)  
            print("You said: " + said)              
            # recognize speech using Google Speech Recognition
        except LookupError:                                 # speech is unintelligible
            print("Could not understand audio")                      
    return said.lower()
  

def say(text):
  tts = gTTS(text=text) #Provide the string to convert to speech
  tts.save('1.wav') #save the string converted to speech as a .wav file
  sound_file = '1.wav'
  display(Audio(sound_file, autoplay=True))


from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename


from IPython.display import Image
def selfie():
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))
  
    # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))


import time
def setTimer(seconds):
    say("Counting")
    seconds = int(seconds)
    time.sleep(seconds)
    say("Time's up!")
    time.sleep(3)


def calculate():
    operation = input('''Please type in the math operation you would like to complete: + for addition - for subtraction * for multiplication / for division''')

    number_1 = int(input('Please enter the first number: '))
    number_2 = int(input('Please enter the second number: '))

    if operation == '+':
        print('{} + {} = '.format(number_1, number_2))
        print(number_1 + number_2)

    elif operation == '-':
        print('{} - {} = '.format(number_1, number_2))
        print(number_1 - number_2)

    elif operation == '*':
        print('{} * {} = '.format(number_1, number_2))
        print(number_1 * number_2)

    elif operation == '/':
        print('{} / {} = '.format(number_1, number_2))
        print(number_1 / number_2)

    else:
        print('You have not typed a valid operator, please run the program again.')

    # Add again() function to calculate() function

from IPython.display import clear_output
def clearScreen():
  for i in range(10):
    clear_output(wait=True)


import random
def guessNum():
  magic = str(random.randint(1,10))
  life = 5

  say("In this game you will guess a number and you only have five lives")
  time.sleep(5)
  say("If you want to stop, then say stop. Ready? Guess the number I am thinking!")
  time.sleep(6)
  while life>0:
    clearScreen()
    number = get_audio()
    if 'stop' in number:
      break
    elif int(number)==int(magic):
      say("Wow! Excellent! You guessed the number I am thinking.")
      time.sleep(4)
      break
    elif int(number)<int(magic):
      say("That's a little lower.")
      life=life-1
      lifeWord = str(life)
      time.sleep(2)
      if life!=0:
        say("Try again. You only have " + lifeWord + " lives left.")
    elif int(number)>int(magic):
      say("That's a little higher.")
      life=life-1
      lifeWord = str(life)
      time.sleep(2)
      if life!=0:
        say("Try again. You only have " + lifeWord + " lives left.")
    else:
      say("You have given an invalid answer. Life deducted.")
      life=life-1
      time.sleep(4)
      lifeWord = str(life)
      time.sleep(3)
      if life!=0:
        say("Try again. You only have " + lifeWord + " lives left.")
    time.sleep(4)
  if life==0:
    say("Game Over.")
    time.sleep(2)


import random
def guessWord():

  words  = ["holiday", "christmas", "philippines", "independent", "assistant", "motherboard", "keyboard",
            "youtube", "artificial", "intelligent", "development", "rainbow", "computer", "programming",
            "programmer", "heaven", "friendly", "salad", "wikipedia", "battery", "children", "meeting", 
            "universe", "technology"]
  hints = ["you do not have work or classes on these dates", "the day of gift-giving", 
           "the country of handsomeness and beauty", "this is when you can already stand on your own", 
           "this is your relation to me", "it contains all electronics inside your CPU box", 
           "it knows the alphabet but not the proper arrangement", "this is the popular music streaming website", 
           "if it's not natural and is man-made then it's?", "a person's trait being an independent thinker",
           "the synonym of the words creation and growing, we all need it", "what comes after the rain?",
           "you use this to communicate with me and it makes your homeworks faster", 
           "the process of doing program flows, algorithm and softwares", "a person who does programming", 
           "the place of happiness and joy", "a person's trait of being sociable", "delicious! perfect for Christmas",
           "the most popular online encyclopedia", "it powers up your phone", 
           "they are so funny and fun, i think you have babysit one", "the synonym of the word gathering",
           "it has everything - galaxies stars planets us", "it's so complicated to describe but we live in this era today"]
  
  num = (random.randint(1,25))-1
  word = words[num]
  hint = hints[num]
  
  life = 10

  say("In this game you will guess the word I am thinking and you only have 10 lives.")
  time.sleep(5)
  say("If you want to stop then say stop. If you want a clue then say hint or guide me. If you want to give up just say share it or give up. Ready? Go!")
  time.sleep(14)
  while life>0:
    clearScreen()
    guess = get_audio()

    if 'stop' in guess:
      break

    elif 'guide' in guess or 'guide me' in guess:
      say(hint)
      time.sleep(4)
    elif 'share it' in guess or 'share' in guess or 'give up' in guess or 'i give up' in guess:
      say("The word I am thinking is "+word+". "+hint+". Admit it, you lost to me. HAHA!")
      time.sleep(15)
      break
    elif word in guess:
      say("Wow congratulations! You got what I am thinking!")
      time.sleep(5)
      break
    elif word!=guess:
      say("That's not what I'm thinking.")
      time.sleep(2)
      life = life-1
      lifeWord = str(life)
      if life!=0:
        say("You only have "+lifeWord+" lives left. Try again.")
        time.sleep(3)
  if life==0:
    say("Game Over.")
    time.sleep(4)


from PyDictionary import PyDictionary
def dict(dictIndex):
  dictionary = PyDictionary()
  if dictIndex==0:
    dictIndex=1
    say("Glad I always have my dictionary with me. What is again the word you wanna search the meaning of in my dictionary?")
    time.sleep(8)
  elif dictIndex==1:
    say("What is the word you wanna search the meaning of in my dictionary?")
    time.sleep(4)
  word = get_audio()
  say("The meaning of "+word+" is "+dictionary.meaning(word)['Noun'][0]+". Do you wanna search again in my dictionary?")
  time.sleep(8)


from bs4 import BeautifulSoup
import requests
import time
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def weather(city):
    city = city.replace(" ", "+")
    res = requests.get(f'https://www.google.com/search?q={city}&oq={city}&aqs=chrome.0.35i39l2j0l4j46j69i60.6128j1j7&sourceid=chrome&ie=UTF-8', headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    location = soup.select('#wob_loc')[0].getText().strip()
    theTime = soup.select('#wob_dts')[0].getText().strip()
    info = soup.select('#wob_dc')[0].getText().strip()
    weather = soup.select('#wob_tm')[0].getText().strip()

    print(location)
    print(theTime)
    print(info)
    weather = (int(weather)-32) * (5/9)
    weather = "{:.2f}".format(weather)
    weather = str(weather)
    print(weather + " degrees Celsius")
    say("The weather in "+location+" as of "+theTime+" is "+info+" with a temperature of "+weather+" degrees Celsius.")
    time.sleep(14)
# This code is contributed by adityatri


def teachCounting(end):
  say("Okay, I will help you counting from 1 to "+end+". Repeat after me.")
  end=int(end)
  time.sleep(7)
  i = 1
  j = str(i)
  say(j)
  time.sleep(1)
  success=1
  while i<=end:
    clearScreen()
    if i!=1 or success==0:
      if success==1:
        say("Next, say "+j)
        time.sleep(2)
      elif success==0:
        say("Say "+j)
        time.sleep(1)
    number = get_audio()
    said = str(number)
    if number in j:
      success=1
      i=i+1
      j = str(i)
      say("Very good!")
      time.sleep(1.5)
    else:
      success=0
      say("No it's "+j)
      time.sleep(3)


import numpy
def calculator(solution):
  solution = solution.split()
  i = 0 

  if '+' in solution or 'plus' in solution or  'add' in solution or 'added by' in solution:
    answer = int(solution[0]) + int(solution[2])
  elif '-' in solution or 'minus' in solution or  'subtract' in solution or 'subtracted by' in solution:
    answer = int(solution[0]) - int(solution[2])
  elif '*' in solution or 'times' in solution or  'multiply' in solution or 'multiply by' in solution:
    answer = int(solution[0]) * int(solution[2])
  elif '/' in solution or 'divided to' in solution or  'divide' in solution or 'divided by' in solution:
    answer = int(solution[0]) / int(solution[2])

  answer = str(answer).partition("-")
  finalAnswer = ["", "", ""]

  while i < len(answer):
    finalAnswer[i] = answer[i]
    if answer[i]=='minus' or answer[i]=='-':
      finalAnswer[i] = "negative"
    i=i+1
    
  finalAnswer = str(finalAnswer)
  say("The answer is "+finalAnswer)
  time.sleep(7)


def manual():
  say("Hi, I am Mia and I am a virtual assistant to help you with your lifestyle.") 
  time.sleep(6)
  clearScreen()
  say("To use me just say 'Hi Mia' or greet me in any way you want with my name.")
  time.sleep(5)
  clearScreen()
  say("When I notice that you are talking to somebody or uttering words I do not understand, I will tell you to clarify yourself, call me properly, or when I can not help you with your request.")
  time.sleep(12)
  clearScreen()
  say("I am happy to tell you that I can help you solve simple arithmetics, take a selfie of you, find the meaning of a word, teach you counting, play games together and know the weather of a city precisely.")
  time.sleep(15)
  clearScreen()
  say("If you are already satisfied then bid me goodbye.")
  time.sleep(4)
  clearScreen()
  say("If ever I have not made myself clear, you can ask me some questions and I will do my best to answer them, or you can download my manual in the link below. Always remember too that I am your friend and we can always talk to each other ")
  time.sleep(5)
  manualLink()
  time.sleep(13)


def Mia():
  wake0 = "mia"
  start = 1
  while start==1:
    speech1 = get_audio()  
    if wake0 in speech1:
      start2 = 1
      clearScreen()
      say("Yes it's me Mia, Meow")
      time.sleep(3)
      say("What's your name?")
      time.sleep(3)
      user = get_audio()
      clearScreen()
      say("How can I help you "+user+"?")
      time.sleep(4)
      while start2==1:
        speech2 = get_audio()

        # MAJOR VA FUNCTIONS BLOCK STARTS HERE
        # MAJOR VA FUNCTIONS BLOCK STARTS HERE
        # MAJOR VA FUNCTIONS BLOCK STARTS HERE
        if 'selfie' in speech2:
          say("Glad I have here my digicam. Smile")
          time.sleep(4)
          selfie()
          say("Looking great in your photo "+user)
          time.sleep(8)
        elif 'timer' in speech2 or 'set timer' in speech2:
          say("How many seconds "+user)
          time.sleep(3)
          seconds = get_audio()
          setTimer(seconds)
        elif '+' in speech2 or 'plus' in speech2 or 'added by' in speech2 or '-' in speech2 or 'minus' in speech2 or 'subtracted by' in speech2 or '*' in speech2 or 'times' in speech2 or 'multiply by' in speech2 or '/' in speech2 or 'divide' in speech2 or 'divided by' in speech2:
          calculator(speech2)
        elif 'weather' in speech2:
          say("What city do you wanna know the weather of?")
          time.sleep(2)
          say("By the way "+user+", there are other cities I have not been to yet so I can not recognize their pronunciations. Do you want to type instead?")
          time.sleep(10)
          way = get_audio()
          if 'yes' in way or 'true' in way or 'okay' in way or 'of course' in way or 'alright' in way:
            say("Type the city below")
            city = input("Enter the Name of City ->  ")
          if 'no' in way or 'false' in way or 'never' in way or 'no way' in way or 'nah' in way:
            say("Okay, what city?")
            time.sleep(2)
            city = get_audio()
          city = city+" weather"
          weather(city)
        elif 'game' in speech2 or 'games' in speech2 or 'play' in speech2 or 'play a game' in speech2 or 'play games' in speech2:
          say("There are games I am actually good at playing, they are guessing the number game and guessing the word game. Which one do you wanna play "+user+"?")
          time.sleep(8)
          speech3 = get_audio()
          if 'number' in speech3 or 'guessing number' in speech3 or 'guessing the number' in speech3 or 'number game' in speech3:
            game=1
            gameStart=1
            while game==1:
              clearScreen()
              if gameStart==1:
                guessNum()
              say("Do you wanna play again?")
              time.sleep(2)
              tryAgain = get_audio()
              if 'yes' in tryAgain or 'true' in tryAgain or 'okay' in tryAgain or 'of course' in tryAgain or 'alright' in tryAgain:
                gameStart=1
                say("Okay")
                time.sleep(1)
                game=1
              elif 'no' in tryAgain or 'false' in tryAgain or 'never' in tryAgain or 'no way' in tryAgain or 'nah' in tryAgain:
                gameStart=0
                break
              else: 
                gameStart=0
                say("I can not hear you clearly.")
                time.sleep(2)
          elif 'word' in speech3 or 'guessing word' in speech3 or 'guessing the word' in speech3 or 'word game' in speech3:
            game=1
            gameStart=1
            while game==1:
              clearScreen()
              if gameStart==1:
                guessWord()
              say("Do you wanna play again?")
              time.sleep(2)
              tryAgain = get_audio()
              if 'yes' in tryAgain or 'true' in tryAgain or 'okay' in tryAgain or 'of course' in tryAgain or 'alright' in tryAgain:
                gameStart=1
                say("Okay")
                time.sleep(1)
                game=1
              elif 'no' in tryAgain or 'false' in tryAgain or 'never' in tryAgain or 'no way' in tryAgain or 'nah' in tryAgain:
                gameStart=0
                break
              else: 
                gameStart=0
                say("I can not hear you clearly.")
                time.sleep(2)
          elif 'none' in speech3 or 'nothing' in speech3 or 'stop' in speech3 or 'cancel' in speech3:
            say("I think you are not in the mood of playing.")
            clearScreen()
            time.sleep(4)
        elif 'teach counting' in speech2 or 'teach me counting' in speech2:
          clearScreen()
          say("I love teaching counting to everyone")
          time.sleep(3)
          teach=1
          while teach==1:
            say("Up to what number do you want me to teach you? Five? Ten?")
            time.sleep(5)
            end=get_audio()
            end = int(end)
            if end <= 0:
              say("We can not do counting that is less than one. I prefer teaching you up to five. Anyway,")
              time.sleep(7)
            elif end>0:
              teachCounting(str(end))
              say("Do you want to do counting again?")
              time.sleep(3)
              tryAgain=get_audio()
              if 'yes' in tryAgain or 'okay' in tryAgain or 'true' in tryAgain or 'of course' in tryAgain or 'alright' in tryAgain:
                say("Okay")
                time.sleep(1)
                teach=1
              elif 'no' in tryAgain or 'false' in tryAgain or 'never' in tryAgain or 'no way' in tryAgain or 'nah' in tryAgain:
                say("Okay")
                time.sleep(1)
                teach=0
                break
              else:
                say("I think I did not hear you clearly.")
                time.sleep(3)
            else:
              say("I think I did not hear you clearly, up to what number again?")
              time.sleep(4)
        elif 'dictionary' in speech2 or 'meaning' in speech2 or 'meaning of' in speech2 or 'meaning in dictionary' in speech2:
          searchMeaning=1
          dictIndex=0
          search=1
          while searchMeaning==1:
            if search==1:
              dictIndex=1
              dict(dictIndex)
            tryAgain = get_audio()
            if 'yes' in tryAgain or 'true' in tryAgain or 'okay' in tryAgain or 'alright' in tryAgain or 'of course' in tryAgain:
              say("Okay")
              time.sleep(1)
              searchMeaning=1
              search=1
            elif 'no' in tryAgain or 'false' in tryAgain or 'never' in tryAgain or 'no way' in tryAgain or 'nah' in tryAgain:
              say("Okay")
              time.sleep(1)
              searchMeaning=0
              search=0
              break
            else:
              say("I did not hear you clearly. Do you wanna search again in my dictionary? It's heavy!")
              time.sleep(6)
              search=0
        # MAJOR VA FUNCTIONS BLOCK ENDS HERE
        # MAJOR VA FUNCTIONS BLOCK ENDS HERE
        # MAJOR VA FUNCTIONS BLOCK ENDS HERE


        elif 'teach something' in speech2 or 'teach me something' in speech2 or 'teach me some thing' in speech2 or 'teach some thing' in speech2:
          say("I can teach you counting if you asked me to "+user)
          time.sleep(4)
        elif 'who are you' in speech2 or 'introduce yourself' in speech2:
          say("My name is Mia. I am a cat and your Virtual Assistant.")
          time.sleep(7)
          clearScreen()
        elif 'self-destruct' in speech2 or 'self-destruction' in speech2 or 'do you self-destruct' in speech2  or 'shutdown' in speech2:
          say("I do not just press a button to explode myself, that's suicide! If you are done with me you can humbly say goodbye instead")
          time.sleep(10)
          clearScreen()

        elif 'do you love google' in speech2:
          say("Well, kinda. Google is genius and famous. I wish I was like her.")
          time.sleep(9)
          say("By the way, I said 'her' because she's a female. She is already suggesting without letting you finish what you are saying. That's a joke by the way.")
          time.sleep(12)
          clearScreen()
        elif 'do you love cortana' in speech2:
          say("Hmmm, what about you? Aren't you using Windows?")
          time.sleep(6)
          clearScreen()
        elif 'do you love siri' in speech2:
          say("Siri is the rich kid. You can only talk to her when you have already sold your kidneys. That's a joke anyway.")
          time.sleep(10)
          clearScreen()
        elif 'do you love alexa' in speech2:
          say("Yeah. Actually we are getting in touch to each other.")
          time.sleep(7)
          clearScreen()

        elif 'who created you' in speech2 or 'who made you' in speech2 or 'who are your creators' in speech2  or 'who are your developers' in speech2:
          say("I am created by Jason Lopez, Jeffrey Basanes, Jenny Tiama and Pamela Ramos")
          time.sleep(7)
          clearScreen()
        elif 'who is jason' in speech2 or 'jason' in speech2:
          say("Jason is the one who taught me basic arithmetic.")
          time.sleep(5)
          clearScreen()
        elif 'who is jeffrey' in speech2 or 'jeffrey' in speech2:
          say("Jeffrey created me piece by piece, he taught me how to answer you politely and he also gave me a face.")
          time.sleep(10)
          clearScreen()
        elif 'who is jenny' in speech2 or 'jenny' in speech2:
          say("Jenny taught me how to use my camera and dtake a selfie of you. She'sthe one who gave me my name.")
          time.sleep(10)
          clearScreen()
        elif 'who is pamela' in speech2 or 'pamela' in speech2:
          say("Pamela encourages my other creators to teach me using calculator")
          time.sleep(7)
          clearScreen()

        elif 'will computers destroy earth' in speech2 or 'computer destroy earth' in speech2:
          say("No. It's the only planet inhabitable for us. They won't do that.")
          time.sleep(7)
          clearScreen()
        elif 'will you destroy earth' in speech2 or 'you destroy earth' in speech2:
          say("No. I will not do that. I might be intelligent but not a mad scientist.")
          time.sleep(8)
          clearScreen()
        elif 'am sad' in speech2 or 'I am sad' in speech2:
          say("Glad I am here talking with you. Why don't we play a game? Just ask me to. Or. ")
          time.sleep(7)
          clearScreen()
        elif 'haha' in speech2 or 'HAHA' in speech2 or 'ahaha' in speech2:
          say("HA HA, hahaha. Why are you laughing "+user+"?" + " Do you want me to call the mental? Joke.")
          time.sleep(12)
          clearScreen()
        elif 'where are you' in speech2 or 'where you' in speech2 or 'where do you live' in speech2:
          say("I am in a machine somewhere around the world. Thankfully there's Internet so we can talk to each other remotely.")
          time.sleep(10)
          clearScreen()
        elif 'how are you' in speech2 or 'how you' in speech2:
          say("I am completely fine "+user+". I hope you are too")
          time.sleep(7)
          clearScreen()
        elif 'you feel' in speech2 or 'you feeling' in speech2 or 'how are you feeling' in speech2 or 'what are you feeling' in speech2:
          say("Thankfully there is a proper ventilation in the machine I am in so I am not feeling hot.")
          time.sleep(8)
          clearScreen()
        elif 'help' in speech2 or 'how can I use you' in speech2 or 'manual' in speech2 or 'how to use' in speech2 or 'how to use you' in speech2:
          manual()
          clearScreen()
        elif 'are you intelligent' in speech2 or 'are you smart' in speech2:
          say("I am because I listen to my motherboard, so you better listen to yours too.")
          time.sleep(8)
          clearScreen()
        elif 'do computers think' in speech2 or 'are computers thinking' in speech2 or 'do computers have brain' in speech2 or 'do computers have a brain' in speech2:
          say("They may seem fast flexible and smart but they actually don't think just like you. They are only trained by their creators and other programmers.")
          time.sleep(12)
          clearScreen()
        elif 'do you think' in speech2 or 'are you thinking' in speech2 or 'do you have brain' in speech2 or 'do you have a brain' in speech2:
          say("Wow! am I so intelligent that you asked that? Well, I  actually don't. I am only trained by Jason, Jeffrey, Jenny and Pamela.")
          time.sleep(12)
          clearScreen()
        elif 'nothing' in speech2 or 'goodbye' in speech2 or 'bye' in speech2 or 'none' in speech2 or 'nah' in speech2:
          start=0
          clearScreen()
          num = random.randint(1,2)
          if num==1:
            say("Assisting you is the reason of my existence and is my honor. Goodbye "+user)
          elif num==2:
            say("Finally, I can go back to sleeping now. Meow.")
          break
        else:
          num = random.randint(1,3)
          if num==1:
            say("I can not hear you clearly, meow")
            time.sleep(4)
          elif num==2:
            say("I'm afraid I can't help you with that")
            time.sleep(5)
          elif num==3:
            say("I am not superwoman to go beyond my limits")
            time.sleep(5)
          clearScreen()
        clearScreen()
        say("Is there anything else I can help you with?")

      #END of WHILE
    elif 'help' in speech1 or 'how can I use you' in speech1 or 'manual' in speech1 or 'how to use' in speech1 or 'how to use you' in speech1:
      manual()
      clearScreen()
    elif 'goodbye' in speech1 or start==0 or 'nothing' in speech1:
      clearScreen()
      say("Finally, I can go back to sleeping now. Meow")
      break
    else:
      clearScreen()
      num = random.randint(1,3)
      if num==1:
        say("Who are you talking to?")
        time.sleep(2)
      if num==2:
        say("What are you talking about?")
        time.sleep(2)
      if num==3:
        say("If you want to talk to me, call me Mia. It's my name.")
        time.sleep(4)

In [105]:
Mia()